# Large-Scale Quantum Simulation with MLX

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Year 1-2, Semesters 1A-2A (Days 001-168)
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

## Overview: Pushing Qubit Counts on Apple Silicon

State-vector simulation is the gold standard for exact quantum simulation:
every amplitude is tracked, every gate is applied as a linear map.
The challenge is **exponential memory**: $n$ qubits require $2^n$ complex
amplitudes.

Apple Silicon's **unified memory** gives us an edge:
- No CPU-to-GPU copy overhead
- Up to 512 GB on Mac Studio Ultra
- MLX's lazy evaluation minimizes peak memory

In this notebook we build a **complete quantum circuit simulator** backed
by MLX, then stress-test it from 10 to 28+ qubits.

In [ ]:
# --- Imports and MLX setup ---
import time
import sys
import os
import numpy as np

try:
    import mlx.core as mx
    HAS_MLX = True
    print("MLX available --- Apple Silicon acceleration enabled")
except ImportError:
    HAS_MLX = False
    print("MLX not available --- falling back to NumPy")

# Hardware config
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits, QUBIT_LIMITS
    SAFE_QUBITS = get_max_qubits("safe")
    MAX_QUBITS = get_max_qubits("max")
    DEMO_QUBITS = get_max_qubits("demo")
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB")
    print(f"Qubit limits: {SAFE_QUBITS} (safe) / {MAX_QUBITS} (max) / {DEMO_QUBITS} (demo)")
except ImportError:
    SAFE_QUBITS = 22
    MAX_QUBITS = 25
    DEMO_QUBITS = 18
    print("hardware_config not found --- using conservative defaults")

## Memory Analysis: How Many Qubits Can We Simulate?

Each qubit doubles the state vector size:

$$\text{Memory} = 2^n \times B \text{ bytes}$$

where $B = 16$ for `complex128` (double precision) or $B = 8$ for `complex64`.

| Qubits | Amplitudes | complex128 | complex64 |
|--------|-----------|------------|-----------|
| 10 | 1,024 | 16 KB | 8 KB |
| 20 | 1,048,576 | 16 MB | 8 MB |
| 25 | 33,554,432 | 512 MB | 256 MB |
| 28 | 268,435,456 | 4 GB | 2 GB |
| 30 | 1,073,741,824 | 16 GB | 8 GB |
| 33 | 8,589,934,592 | 128 GB | 64 GB |

On a **128 GB MacBook Pro M4 Max**, we can comfortably fit a 30-qubit
`complex128` state vector (16 GB) with room for gate operations.
A **512 GB Mac Studio** can push to 33 qubits.

In [ ]:
# --- Memory requirement calculator ---
import numpy as np

def memory_for_qubits(n, dtype="complex128"):
    """Calculate memory needed for an n-qubit state vector."""
    bytes_per_element = 16 if dtype == "complex128" else 8
    total_bytes = (2 ** n) * bytes_per_element
    return total_bytes

def format_bytes(b):
    """Human-readable byte size."""
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if b < 1024:
            return f"{b:.1f} {unit}"
        b /= 1024
    return f"{b:.1f} PB"

print("Quantum Simulation Memory Requirements")
print("=" * 65)
print(f"{'Qubits':>7} | {'Amplitudes':>15} | {'complex128':>12} | {'complex64':>12} | {'Feasible?':>10}")
print("-" * 65)

try:
    available_gb = HARDWARE["memory_gb"]
except NameError:
    available_gb = 16

for n in range(5, 36):
    dim = 2 ** n
    mem_128 = memory_for_qubits(n, "complex128")
    mem_64 = memory_for_qubits(n, "complex64")
    # Need ~3x state vector for operations (input + output + gate overhead)
    feasible = "YES" if mem_128 * 3 < available_gb * 1024**3 else "no"
    marker = " <-- limit" if feasible == "no" and memory_for_qubits(n-1, "complex128") * 3 < available_gb * 1024**3 else ""
    print(f"{n:>7} | {dim:>15,} | {format_bytes(mem_128):>12} | {format_bytes(mem_64):>12} | {feasible:>10}{marker}")

## MLX Quantum Circuit Simulator: Design

Our simulator avoids building full $2^n \times 2^n$ gate matrices.
Instead, we use the **tensor product structure** to apply gates efficiently:

**Single-qubit gate on qubit $k$ of $n$-qubit state:**
1. Reshape state: $(2^n,) \to (2^k, 2, 2^{n-k-1})$
2. Apply $2 \times 2$ gate via tensor contraction on axis 1
3. Flatten back to $(2^n,)$

This uses $O(2^n)$ memory instead of $O(2^{2n})$.

**CNOT (controlled gate):**
1. Reshape into multi-index form
2. Conditionally apply target gate when control qubit is $|1\rangle$

This approach is how real state-vector simulators (Qiskit Aer, Cirq) work.

In [ ]:
# --- MLX Quantum Circuit Simulator ---
import numpy as np
import time

class MLXQuantumSimulator:
    """State-vector quantum circuit simulator with MLX backend.

    Uses efficient tensor-contraction for gate application instead
    of full 2^n x 2^n matrix construction.
    """

    def __init__(self, n_qubits, use_mlx=True, dtype="complex128"):
        self.n = n_qubits
        self.dim = 2 ** n_qubits
        self.use_mlx = use_mlx and HAS_MLX
        self.dtype = dtype
        self.gate_log = []  # track applied gates

        # Initialize |00...0>
        if self.use_mlx:
            np_state = np.zeros(self.dim, dtype=np.complex128 if dtype == "complex128" else np.complex64)
            np_state[0] = 1.0
            self.state = mx.array(np_state)
            mx.eval(self.state)
        else:
            self.state = np.zeros(self.dim, dtype=np.complex128 if dtype == "complex128" else np.complex64)
            self.state[0] = 1.0

    def _to_np(self, arr):
        if self.use_mlx and isinstance(arr, mx.array):
            return np.array(arr)
        return np.asarray(arr)

    def _from_np(self, arr):
        if self.use_mlx:
            return mx.array(arr)
        return arr

    def apply_single_qubit_gate(self, gate_np, target):
        """Apply a 2x2 gate to qubit `target` using reshape trick.

        Reshapes (2^n,) -> (2^target, 2, 2^(n-target-1)) then contracts
        the gate along axis 1.
        """
        state_np = self._to_np(self.state)
        n = self.n
        # Reshape to isolate target qubit
        shape = [2] * n
        state_r = state_np.reshape(shape)

        # Apply gate: contract along target axis
        # np.tensordot(gate, state, axes=([1], [target])) then move axis back
        result = np.tensordot(gate_np, state_r, axes=([1], [target]))
        # tensordot puts the gate's output axis first; move it to position `target`
        result = np.moveaxis(result, 0, target)
        self.state = self._from_np(result.reshape(self.dim))
        if self.use_mlx:
            mx.eval(self.state)
        self.gate_log.append(("single", target))

    def apply_cnot(self, control, target):
        """Apply CNOT gate: flip target if control is |1>."""
        state_np = self._to_np(self.state)
        n = self.n
        shape = [2] * n
        state_r = state_np.reshape(shape)

        # X gate for the target, conditioned on control=1
        x_gate = np.array([[0, 1], [1, 0]], dtype=state_np.dtype)

        # Extract slice where control qubit = 1
        slices_1 = [slice(None)] * n
        slices_1[control] = 1
        sub = state_r[tuple(slices_1)]

        # Apply X to target qubit in that subspace
        result = state_r.copy()
        sub_shape = list(sub.shape)
        # target axis in sub-array (shifted if target > control)
        t_ax = target if target < control else target - 1
        sub_result = np.tensordot(x_gate, sub, axes=([1], [t_ax]))
        sub_result = np.moveaxis(sub_result, 0, t_ax)

        result[tuple(slices_1)] = sub_result
        self.state = self._from_np(result.reshape(self.dim))
        if self.use_mlx:
            mx.eval(self.state)
        self.gate_log.append(("cnot", control, target))

    def apply_controlled_gate(self, gate_np, control, target):
        """Apply arbitrary controlled-U gate."""
        state_np = self._to_np(self.state)
        n = self.n
        shape = [2] * n
        state_r = state_np.reshape(shape)

        slices_1 = [slice(None)] * n
        slices_1[control] = 1

        sub = state_r[tuple(slices_1)]
        result = state_r.copy()
        t_ax = target if target < control else target - 1
        sub_result = np.tensordot(gate_np, sub, axes=([1], [t_ax]))
        sub_result = np.moveaxis(sub_result, 0, t_ax)

        result[tuple(slices_1)] = sub_result
        self.state = self._from_np(result.reshape(self.dim))
        if self.use_mlx:
            mx.eval(self.state)
        self.gate_log.append(("controlled", control, target))

    def h(self, target):
        """Hadamard gate."""
        h = np.array([[1, 1], [1, -1]], dtype=np.complex128) / np.sqrt(2)
        self.apply_single_qubit_gate(h, target)

    def x(self, target):
        """Pauli-X gate."""
        x = np.array([[0, 1], [1, 0]], dtype=np.complex128)
        self.apply_single_qubit_gate(x, target)

    def z(self, target):
        """Pauli-Z gate."""
        z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
        self.apply_single_qubit_gate(z, target)

    def rz(self, target, theta):
        """Rz rotation gate."""
        rz = np.array([
            [np.exp(-1j * theta / 2), 0],
            [0, np.exp(1j * theta / 2)]
        ], dtype=np.complex128)
        self.apply_single_qubit_gate(rz, target)

    def cnot(self, control, target):
        """CNOT gate."""
        self.apply_cnot(control, target)

    def measure(self, n_shots=1024, seed=42):
        """Simulate measurement."""
        probs = np.abs(self._to_np(self.state)) ** 2
        probs = probs / probs.sum()
        rng = np.random.default_rng(seed)
        outcomes = rng.choice(self.dim, size=n_shots, p=probs)
        counts = {}
        for o in outcomes:
            label = format(o, f"0{self.n}b")
            counts[label] = counts.get(label, 0) + 1
        return counts

    def probabilities(self):
        """Return probability distribution."""
        return np.abs(self._to_np(self.state)) ** 2

    def memory_usage_mb(self):
        """Estimated memory usage in MB."""
        bytes_per = 16 if self.dtype == "complex128" else 8
        return self.dim * bytes_per / (1024 ** 2)

print("MLXQuantumSimulator class defined")
print(f"Backend: {'MLX (Metal GPU)' if HAS_MLX else 'NumPy (CPU)'}")

## Testing the Simulator: Bell State Verification

Let us verify our simulator produces correct results with a simple test:
create a Bell state and check the measurement statistics.

In [ ]:
# --- Test: Bell state creation ---
sim = MLXQuantumSimulator(2)
sim.h(0)
sim.cnot(0, 1)

probs = sim.probabilities()
counts = sim.measure(n_shots=10000)

print("Bell State |Phi+> Test")
print("=" * 50)
print(f"State vector: {sim._to_np(sim.state)}")
print(f"\nProbabilities:")
for i, p in enumerate(probs):
    if p > 1e-10:
        print(f"  |{i:02b}> : {p:.6f}")

print(f"\nMeasurement (10k shots):")
for label in sorted(counts.keys()):
    print(f"  |{label}> : {counts[label]} ({counts[label]/10000:.4f})")

# Verify
expected = np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], dtype=np.complex128)
actual = sim._to_np(sim.state)
assert np.allclose(actual, expected, atol=1e-10), "Bell state mismatch!"
print("\nBell state VERIFIED")

## GHZ State Creation: Scaling Entanglement

The **GHZ (Greenberger-Horne-Zeilinger) state** is the $n$-qubit generalization
of the Bell state:

$$|GHZ_n\rangle = \frac{1}{\sqrt{2}}(|00\cdots0\rangle + |11\cdots1\rangle)$$

Construction:
1. Apply Hadamard to qubit 0
2. Apply CNOT from qubit 0 to every other qubit

This creates maximal entanglement across all $n$ qubits.

In [ ]:
# --- GHZ state creation and verification ---
def create_ghz(n_qubits, use_mlx=True):
    """Create an n-qubit GHZ state and return the simulator."""
    t0 = time.perf_counter()
    sim = MLXQuantumSimulator(n_qubits, use_mlx=use_mlx)
    sim.h(0)
    for i in range(1, n_qubits):
        sim.cnot(0, i)
    elapsed = time.perf_counter() - t0
    return sim, elapsed

# Test GHZ for small sizes
for n in [3, 5, 8, 10]:
    sim, t = create_ghz(n)
    probs = sim.probabilities()
    p_000 = probs[0]
    p_111 = probs[-1]
    print(f"GHZ-{n:>2}: P(|{'0'*n}>)={p_000:.6f}  P(|{'1'*n}>)={p_111:.6f}  "
          f"sum={probs.sum():.6f}  time={t*1000:.2f}ms  mem={sim.memory_usage_mb():.2f}MB")

# Detailed view of GHZ-3
sim3, _ = create_ghz(3)
print(f"\nGHZ-3 state vector: {sim3._to_np(sim3.state)}")
counts = sim3.measure(n_shots=10000)
print(f"Measurement (10k shots):")
for label in sorted(counts.keys()):
    print(f"  |{label}> : {counts[label]:>5}")

## Quantum Fourier Transform (QFT)

The **Quantum Fourier Transform** maps computational basis states to
frequency-domain states. It is the quantum analogue of the discrete Fourier
transform and a key component of Shor's algorithm.

$$QFT|j\rangle = \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} e^{2\pi i jk / 2^n} |k\rangle$$

Implementation uses:
1. Hadamard gates
2. Controlled phase rotation gates $R_k = \text{diag}(1, e^{2\pi i / 2^k})$
3. SWAP gates at the end

In [ ]:
# --- Quantum Fourier Transform ---
import numpy as np

def apply_qft(sim):
    """Apply QFT to all qubits of the simulator in-place."""
    n = sim.n
    for i in range(n):
        sim.h(i)
        for j in range(i + 1, n):
            k = j - i + 1
            # Controlled R_k gate
            phase = 2 * np.pi / (2 ** k)
            rk = np.array([
                [1, 0],
                [0, np.exp(1j * phase)]
            ], dtype=np.complex128)
            sim.apply_controlled_gate(rk, j, i)

    # Swap qubits for correct ordering
    for i in range(n // 2):
        j = n - 1 - i
        # SWAP = 3 CNOTs
        sim.cnot(i, j)
        sim.cnot(j, i)
        sim.cnot(i, j)

# Test QFT on |1> (should give uniform phases)
n_test = 4
sim_qft = MLXQuantumSimulator(n_test)
sim_qft.x(0)  # prepare |0001>
print(f"Input state |{'0'*(n_test-1)}1> (qubit 0 = |1>)")
print(f"State before QFT: top amplitudes")
state_before = sim_qft._to_np(sim_qft.state)
for i in range(min(8, len(state_before))):
    if abs(state_before[i]) > 1e-10:
        print(f"  |{i:0{n_test}b}> : {state_before[i]:.6f}")

t0 = time.perf_counter()
apply_qft(sim_qft)
qft_time = time.perf_counter() - t0

print(f"\nAfter QFT ({qft_time*1000:.2f} ms):")
state_after = sim_qft._to_np(sim_qft.state)
probs_after = np.abs(state_after) ** 2
print(f"All probabilities equal? {np.allclose(probs_after, 1.0/2**n_test, atol=1e-10)}")
print(f"Probability per state: {probs_after[0]:.6f} (expected {1.0/2**n_test:.6f})")

# Show phases
print(f"\nPhases of first 8 amplitudes:")
for i in range(min(8, len(state_after))):
    amp = state_after[i]
    phase = np.angle(amp) / np.pi
    print(f"  |{i:0{n_test}b}> : |amp|={np.abs(amp):.4f}, phase={phase:.4f} * pi")

## Scale Test: Pushing Qubit Count

Now we stress-test our simulator. We will create GHZ states for increasing
qubit counts and measure both time and memory. The hardware config sets our
safe upper bound.

In [ ]:
# --- Scale test: GHZ state for increasing qubit counts ---
import time

# Determine test range from hardware
scale_qubits = list(range(8, min(SAFE_QUBITS, 28) + 1, 2))
# Add a few key checkpoints
for q in [20, 24, 26, 28]:
    if q not in scale_qubits and q <= SAFE_QUBITS:
        scale_qubits.append(q)
scale_qubits = sorted(set(scale_qubits))

ghz_times_mlx = []
ghz_times_np = []
ghz_memory = []

print(f"Scale test: GHZ state creation from {scale_qubits[0]} to {scale_qubits[-1]} qubits")
print(f"{'Qubits':>7} | {'MLX (ms)':>10} | {'NumPy (ms)':>10} | {'Speedup':>8} | {'Memory':>10}")
print("-" * 60)

for n in scale_qubits:
    mem_mb = (2 ** n) * 16 / (1024 ** 2)
    ghz_memory.append(mem_mb)

    # MLX timing
    if HAS_MLX:
        _, t_mlx = create_ghz(n, use_mlx=True)
    else:
        t_mlx = 0

    # NumPy timing
    _, t_np = create_ghz(n, use_mlx=False)

    ghz_times_mlx.append(t_mlx * 1000)
    ghz_times_np.append(t_np * 1000)

    speedup = t_np / t_mlx if t_mlx > 0 else 1.0
    print(f"{n:>7} | {t_mlx*1000:>10.2f} | {t_np*1000:>10.2f} | {speedup:>7.2f}x | {format_bytes(int(mem_mb * 1024**2)):>10}")

print(f"\nLargest simulation: {scale_qubits[-1]} qubits, {format_bytes(int(ghz_memory[-1] * 1024**2))} state vector")

In [ ]:
# --- Scale test visualization ---
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Time comparison
axes[0].plot(scale_qubits, ghz_times_np, "o-", label="NumPy", linewidth=2)
axes[0].plot(scale_qubits, ghz_times_mlx, "s-", label="MLX", linewidth=2)
axes[0].set_xlabel("Qubits")
axes[0].set_ylabel("Time (ms)")
axes[0].set_title("GHZ State Creation Time")
axes[0].legend()
axes[0].set_yscale("log")
axes[0].grid(True, alpha=0.3)

# Speedup
speedups = [n / m if m > 0 else 1 for n, m in zip(ghz_times_np, ghz_times_mlx)]
axes[1].plot(scale_qubits, speedups, "D-", color="#2ca02c", linewidth=2)
axes[1].set_xlabel("Qubits")
axes[1].set_ylabel("Speedup (x)")
axes[1].set_title("MLX Speedup")
axes[1].axhline(y=1, color="gray", linestyle="--", alpha=0.5)
axes[1].grid(True, alpha=0.3)

# Memory
axes[2].semilogy(scale_qubits, ghz_memory, "^-", color="#9467bd", linewidth=2)
axes[2].set_xlabel("Qubits")
axes[2].set_ylabel("Memory (MB)")
axes[2].set_title("State Vector Memory")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("mlx_labs/scale_test_ghz.png", dpi=150, bbox_inches="tight")
plt.show()
print("Saved: mlx_labs/scale_test_ghz.png")

## Entanglement Entropy

The **von Neumann entanglement entropy** quantifies how entangled subsystem $A$
is with subsystem $B$:

$$S(\rho_A) = -\text{Tr}(\rho_A \log_2 \rho_A)$$

For a bipartition of an $n$-qubit state into subsystems of sizes $n_A$ and $n_B$:
1. Reshape state vector to $(2^{n_A}, 2^{n_B})$ matrix
2. Compute singular values via SVD
3. Entropy from squared singular values (Schmidt coefficients)

$$S = -\sum_i \lambda_i^2 \log_2(\lambda_i^2)$$

- **Product state:** $S = 0$
- **Bell/GHZ state (half-half split):** $S = 1$ bit

In [ ]:
# --- Entanglement entropy computation ---
import numpy as np

def entanglement_entropy(state, n_qubits, partition_size):
    """Compute von Neumann entanglement entropy for a bipartition.

    Args:
        state: state vector (numpy or mlx array)
        n_qubits: total number of qubits
        partition_size: number of qubits in subsystem A
    Returns:
        entropy in bits
    """
    state_np = np.array(state) if not isinstance(state, np.ndarray) else state
    n_a = partition_size
    n_b = n_qubits - partition_size
    dim_a = 2 ** n_a
    dim_b = 2 ** n_b

    # Reshape to matrix and compute SVD
    psi_matrix = state_np.reshape(dim_a, dim_b)
    singular_values = np.linalg.svd(psi_matrix, compute_uv=False)

    # Schmidt coefficients squared
    schmidt_sq = singular_values ** 2
    schmidt_sq = schmidt_sq[schmidt_sq > 1e-15]  # remove numerical zeros

    # Von Neumann entropy
    entropy = -np.sum(schmidt_sq * np.log2(schmidt_sq))
    return entropy

# Test on known states
print("Entanglement Entropy Tests")
print("=" * 60)

# Product state |00> = |0> x |0>
sim_prod = MLXQuantumSimulator(2)
s_prod = entanglement_entropy(sim_prod._to_np(sim_prod.state), 2, 1)
print(f"Product state |00>:  S = {s_prod:.6f} bits (expected 0)")

# Bell state
sim_bell = MLXQuantumSimulator(2)
sim_bell.h(0)
sim_bell.cnot(0, 1)
s_bell = entanglement_entropy(sim_bell._to_np(sim_bell.state), 2, 1)
print(f"Bell state |Phi+>:   S = {s_bell:.6f} bits (expected 1)")

# GHZ-4 with different partitions
sim_ghz4, _ = create_ghz(4)
for p in range(1, 4):
    s = entanglement_entropy(sim_ghz4._to_np(sim_ghz4.state), 4, p)
    print(f"GHZ-4 (A={p}, B={4-p}): S = {s:.6f} bits (expected 1)")

# GHZ scaling
print(f"\nGHZ entropy scaling (half-half partition):")
for n in [4, 6, 8, 10, 12]:
    sim_ghz, t = create_ghz(n)
    s = entanglement_entropy(sim_ghz._to_np(sim_ghz.state), n, n // 2)
    print(f"  GHZ-{n:>2}: S = {s:.6f} bits  (time: {t*1000:.2f} ms)")

## Comparison: MLX Simulator vs Qiskit Aer

If Qiskit is installed, we compare our MLX simulator against Qiskit's
Aer statevector simulator for GHZ state creation. This validates our
results and benchmarks our performance against a production simulator.

Note: Qiskit is optional. The comparison is skipped gracefully if not installed.

In [ ]:
# --- Compare with Qiskit Aer (if available) ---
import time

try:
    from qiskit import QuantumCircuit
    from qiskit_aer import AerSimulator
    HAS_QISKIT = True
    print("Qiskit Aer available for comparison")
except ImportError:
    HAS_QISKIT = False
    print("Qiskit not installed --- skipping comparison")
    print("Install with: pip install qiskit qiskit-aer")

if HAS_QISKIT:
    comparison_qubits = [4, 8, 12, 16, 20]
    if DEMO_QUBITS >= 24:
        comparison_qubits.append(24)

    mlx_times_cmp = []
    qiskit_times_cmp = []

    print(f"\n{'Qubits':>7} | {'MLX (ms)':>10} | {'Qiskit (ms)':>12} | {'Speedup':>8}")
    print("-" * 50)

    for n in comparison_qubits:
        # MLX
        t0 = time.perf_counter()
        sim_mlx, _ = create_ghz(n)
        mlx_t = (time.perf_counter() - t0) * 1000
        mlx_times_cmp.append(mlx_t)

        # Qiskit
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(1, n):
            qc.cx(0, i)
        qc.save_statevector()

        backend = AerSimulator(method="statevector")
        t0 = time.perf_counter()
        result = backend.run(qc, shots=0).result()
        qiskit_t = (time.perf_counter() - t0) * 1000
        qiskit_times_cmp.append(qiskit_t)

        speedup = qiskit_t / mlx_t if mlx_t > 0 else 1
        print(f"{n:>7} | {mlx_t:>10.2f} | {qiskit_t:>12.2f} | {speedup:>7.2f}x")

        # Verify states match
        sv_qiskit = np.array(result.get_statevector(qc))
        sv_mlx = sim_mlx._to_np(sim_mlx.state)
        match = np.allclose(np.abs(sv_mlx), np.abs(sv_qiskit), atol=1e-6)
        if not match:
            print(f"    WARNING: states differ at {n} qubits")
else:
    print("\nSkipping Qiskit comparison. Our simulator results are validated")
    print("by the Bell state and GHZ state tests above.")

In [ ]:
# --- QFT at scale ---
qft_qubits = list(range(4, min(DEMO_QUBITS, 20) + 1, 2))
qft_times = []

print("QFT Performance Scaling")
print("=" * 50)
print(f"{'Qubits':>7} | {'Time (ms)':>10} | {'Memory (MB)':>10}")
print("-" * 40)

for n in qft_qubits:
    sim = MLXQuantumSimulator(n)
    sim.x(0)  # prepare |00...01>

    t0 = time.perf_counter()
    apply_qft(sim)
    t = (time.perf_counter() - t0) * 1000
    qft_times.append(t)

    # Verify: QFT of |1> should give uniform amplitudes
    probs = sim.probabilities()
    uniform = np.allclose(probs, 1.0 / 2**n, atol=1e-8)

    print(f"{n:>7} | {t:>10.2f} | {sim.memory_usage_mb():>10.2f} | {'uniform' if uniform else 'WRONG'}")

# Plot QFT scaling
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(qft_qubits, qft_times, "o-", linewidth=2, color="#d62728")
ax.set_xlabel("Qubits")
ax.set_ylabel("QFT Time (ms)")
ax.set_title("Quantum Fourier Transform Scaling")
ax.set_yscale("log")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("mlx_labs/qft_scaling.png", dpi=150, bbox_inches="tight")
plt.show()
print("Saved: mlx_labs/qft_scaling.png")

In [ ]:
# --- Hardware-adaptive summary ---
print("=" * 60)
print("SIMULATION CAPABILITY SUMMARY")
print("=" * 60)
try:
    print(f"Hardware:         {HARDWARE['chip']}")
    print(f"Memory:           {HARDWARE['memory_gb']} GB unified")
except NameError:
    print("Hardware:         Unknown (hardware_config.py not found)")

print(f"Backend:          {'MLX (Metal GPU)' if HAS_MLX else 'NumPy (CPU only)'}")
print(f"\nQubit limits:")
print(f"  Demo mode:      {DEMO_QUBITS} qubits ({format_bytes(memory_for_qubits(DEMO_QUBITS))} state vector)")
print(f"  Safe mode:      {SAFE_QUBITS} qubits ({format_bytes(memory_for_qubits(SAFE_QUBITS))} state vector)")
print(f"  Maximum:        {MAX_QUBITS} qubits ({format_bytes(memory_for_qubits(MAX_QUBITS))} state vector)")
print(f"\nBenchmark results:")
print(f"  Largest GHZ:    {scale_qubits[-1]} qubits in {ghz_times_mlx[-1]:.1f} ms")
print(f"  Largest QFT:    {qft_qubits[-1]} qubits in {qft_times[-1]:.1f} ms")
print(f"\nRecommendations:")
if SAFE_QUBITS >= 28:
    print("  Your hardware supports research-scale simulations (28+ qubits)")
    print("  Consider complex circuits: VQE, QAOA, error correction codes")
elif SAFE_QUBITS >= 24:
    print("  Your hardware handles most educational simulations well")
    print("  GHZ, QFT, and basic VQE up to ~24 qubits")
else:
    print("  Focus on circuits up to ~20 qubits for interactive work")
    print("  Use reduced qubit counts for algorithm demonstrations")

## Summary

### What we built

- **MLXQuantumSimulator**: a full state-vector simulator using efficient tensor contraction
- **GHZ state** creation scaling to hardware limits
- **Quantum Fourier Transform** implementation
- **Entanglement entropy** measurement
- Performance comparison with Qiskit Aer

### Key insights

1. **Tensor contraction** avoids $O(2^{2n})$ matrix storage --- critical for large simulations
2. **MLX unified memory** eliminates CPU-GPU transfer bottleneck
3. **GHZ states** have exactly 1 bit of entanglement entropy regardless of size
4. **QFT** complexity is $O(n^2)$ gates but each gate costs $O(2^n)$ in state-vector simulation

### Next notebook

In **03_quantum_neural_network.ipynb**, we use this simulator for variational
quantum algorithms: VQE for molecular ground states and quantum kernel methods.